In [1]:
%pylab inline
import pandas as pd
import numpy as np
!nvidia-smi
import sys
print(sys.path)
import py3nvml ## pip install -e git+https://github.com/fbcotter/py3nvml#egg=py3nvml
import subprocess

Populating the interactive namespace from numpy and matplotlib
Thu Jun 22 13:51:18 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 381.22                 Driver Version: 381.22                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 0000:02:00.0      On |                  N/A |
| 23%   41C    P2    70W / 250W |   1151MiB / 11172MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 0000:81:00.0     Off |                  N/A |
| 23%   38C    P8    16W / 250W |      2MiB / 11172MiB |      0%      Default |
+------------------------

In [2]:
import h2oaiglm as h2oaiglm
maxNGPUS = int(subprocess.check_output("nvidia-smi -L | wc -l", shell=True))
print("\nNumber of GPUS:", maxNGPUS)


Loaded H2OAIGLM CPU library

Loaded H2OAIGLM GPU library.

Number of GPUS: 2


In [3]:
!lscpu

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                40
On-line CPU(s) list:   0-39
Thread(s) per core:    2
Core(s) per socket:    10
Socket(s):             2
NUMA node(s):          2
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 79
Model name:            Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz
Stepping:              1
CPU MHz:               1263.796
CPU max MHz:           3100.0000
CPU min MHz:           1200.0000
BogoMIPS:              4401.18
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              25600K
NUMA node0 CPU(s):     0-9,20-29
NUMA node1 CPU(s):     10-19,30-39
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc arch_perfmon pebs bts rep_good nopl xtopo

In [4]:
from os.path import expanduser
home = expanduser("~")
train_file = home + "/h2oai-benchmarks/Data/Homesite/train.csv"
test_file = home + "/h2oai-benchmarks/Data/Homesite/test.csv"

In [5]:
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)

In [6]:
train.drop(['QuoteConversion_Flag'], axis=1,inplace=True)

In [7]:
dataset = pd.concat([train,test], ignore_index = True)
tmp = dataset.dtypes.reset_index().rename(columns = {0:"type"})#["index"]
indx = tmp["type"] == "object"
categoricals = tmp[indx]["index"].tolist()

#replace nans as new category
for col in dataset.columns:
    dataset[col] = dataset[col].fillna("__NA__")

#encode unfreq categories
for col in categoricals:
    val_dict = dataset[col].value_counts()
    val_dict = dataset[col].value_counts().reset_index()
    indx = val_dict[col] < 100
    res = val_dict[indx]["index"].tolist()
    indx = dataset[col].isin(res)
    vals = dataset[col].values
    vals[indx] = "___UNFREQ___"
    dataset[col] = vals
    
#encode all as freqs
for col in categoricals:
    val_dict = dataset[col].value_counts()
    val_dict = val_dict / float(dataset.shape[0])
    val_dict = val_dict.to_dict()
    dataset[col] = dataset[col].apply(lambda x: val_dict[x])
    
#replace nans as new category
for col in dataset.columns:
    dataset[col] = dataset[col].replace("__NA__",0)
    
trainenc = dataset.iloc[:train.shape[0],:].reset_index(drop = True)
testenc = dataset.iloc[train.shape[0]:,:].reset_index(drop = True)

In [9]:
trainencflt = trainenc.values.astype(np.float32)
testencflt = testenc.values.astype(np.float32)
k=1000
rows=np.shape(trainencflt)[0]
print(rows)
np.random.seed(1234)
#labels = np.random.randint(rows, size=rows) % k
import random
import numpy as np
labels=np.asarray([])
num=int(rows/k)
for x in range(0, num+1):
    if x<num:
        many=k
    else:
        many=rows%k
    labels = np.append(labels,np.asarray(random.sample(range(k), many)))
#print(labels)
print(labels.shape)

260753
(260753,)


## H2OAIGLM K-Means (multi-GPU)

In [10]:
n_gpus = 1
model = h2oaiglm.KMeansGPU(n_gpus=n_gpus, k=k, threshold=1e-7, max_iterations=1000)
%time centroids,timefit0,timefit1=model.fit(trainencflt, labels)
print("timefit0=%g timefit1=%g" % (timefit0,timefit1))
#%time train_centroid_distance = model.transform(trainencflt)
#%time train_labels     = model.predict(trainencflt)
#
#%time test_centroid_distance  = model.transform(testencflt)
#%time test_labels     = model.predict(testencflt)

## 44s for 1000 iters
## 48s for 1000 iterations on mr-dl10 (n_gpus=1)
## 41s for 1000 iterations on physics-179.umd.edu (n_gpus=1)

CPU times: user 35.7 s, sys: 9.55 s, total: 45.2 s
Wall time: 46.2 s
timefit0=45.7955 timefit1=45.8383


## H2OAIGLM K-Means (multi-GPU with restrictions)

In [10]:
device=1 # all gpus=0 and pick device=1,2,3...
#model = h2oaiglm.KMeansGPU2(k, init="kmeans++", seed=11, tolerance=1e-2)
#model = h2oaiglm.KMeansGPU2(k, init="random", seed=11, tolerance=1e-2)
model = h2oaiglm.KMeansGPU2(k, init="random", tolerance=1e-10, verbosity=1, device=device)
%time centroids2 = model.fit(trainencflt)
#%time train_centroid_distance2 = model.transform(trainencflt)
#%time train_labels = model.predict(trainencflt)

#%time test_centroid_distance  = model.transform(testencflt)
#%time test_labels     = model.predict(testencflt)

## 60s for 224 iters (stopped early) ==> 268s for 1000 iters
## 51s for 216 iters (stopped early) on mr-dl10
## 49s for 165 iters (stopped early) on physics-179.umd.edu

CPU times: user 41.3 s, sys: 7.68 s, total: 49 s
Wall time: 48.8 s


## H2O

In [11]:
import h2o
h2o.init(strict_version_check=False)
h2otrain = h2o.import_file(train_file)
h2otest = h2o.import_file(test_file)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_131"; Java(TM) SE Runtime Environment (build 1.8.0_131-b11); Java HotSpot(TM) 64-Bit Server VM (build 25.131-b11, mixed mode)
  Starting server from /home/jon/.pyenv/versions/3.6.1/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp5r3h2xfg
  JVM stdout: /tmp/tmp5r3h2xfg/h2o_jon_started_from_python.out
  JVM stderr: /tmp/tmp5r3h2xfg/h2o_jon_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,03 secs
H2O cluster version:,3.11.0.99999
H2O cluster version age:,3 months and 4 days
H2O cluster name:,H2O_from_python_jon_136xjy
H2O cluster total nodes:,1
H2O cluster free memory:,26.67 Gb
H2O cluster total cores:,32
H2O cluster allowed cores:,32
H2O cluster status:,"accepting new members, healthy"
H2O connection url:,http://127.0.0.1:54321
H2O connection proxy:,None


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [12]:
from h2o.estimators.kmeans import H2OKMeansEstimator
h2omodel = H2OKMeansEstimator(k=k, standardize=False, max_iterations=50, init="Random")
%time h2omodel.train(training_frame=h2otrain)

## 600 s for 50 iters -> 12000 secs for 1000 iters

kmeans Model Build progress: |███████████ (cancelled)


H2OJobCancelled: Job<$03017f00000132d4ffffffff$_936b376fca480f693cef1ef663bc4493> was cancelled by the user.

In [ ]:
#%time trainassignments=h2omodel.predict(h2otrain)
#hist(trainassignments.as_data_frame().values)

In [ ]:
#%time testassignments=h2omodel.predict(h2otest)
#hist(testassignments.as_data_frame().values)

## Scikit-Learn

In [10]:
from sklearn.cluster import KMeans
model = KMeans(algorithm='full',n_clusters=k, init='random', n_init=1, n_jobs=-1, max_iter=50, tol=1e-8,verbose=1)
%time model.fit(trainencflt)
#train_assignments = model.predict(trainencflt)
#test_assignments = model.predict(testencflt)

## 58s for 50 iters -> 1160s for 1000 iters
## 111s for 50 iterations (physics-179.umd.edu with defualt pip installed scikit-learn)
## 123s for 50 iterations (physics-179.umd.edu with MKL in scikit-learn)

Initialization complete
Iteration  0, inertia 2494122697011.215
Iteration  1, inertia 1643006533766.852
Iteration  2, inertia 1485651948171.750


Process ForkPoolWorker-58:
Process ForkPoolWorker-41:
Process ForkPoolWorker-47:
Process ForkPoolWorker-38:
Process ForkPoolWorker-59:
Process ForkPoolWorker-50:
Process ForkPoolWorker-34:
Traceback (most recent call last):
Process ForkPoolWorker-52:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Process ForkPoolWorker-42:
Process ForkPoolWorker-51:
Process ForkPoolWorker-61:
Process ForkPoolWorker-36:
Process ForkPoolWorker-56:
Process ForkPoolWorker-63:
Process ForkPoolWorker-60:
Process ForkPoolWorker-54:
Process ForkPoolWorker-55:
Process ForkPoolWorker-64:
Process ForkPoolWorker-37:
Process ForkPoolWorker-40:
Process ForkPoolWorker-62:
Process ForkPoolWorker-39:
Process ForkPoolWorker-53:
Process ForkPoolWorker-44:
Process ForkPoolWorker-46:
Traceback (most recent call last):
Traceback (most recent call 

  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  F

  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/externals/joblib/pool.py", line 360, in get
    racquire()
  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/externals/joblib/pool.py", line 360, in get
    racquire()
  File "/home/jon/.pyenv/versions/3.6.1/lib/python3.6/site-packages/sklearn/externals/joblib/pool.py", line 360, in get
    racquire()


KeyboardInterrupt: 

## TensorFlow

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from tensorflow.contrib.learn import KMeansClustering
from tensorflow.contrib.learn import SKCompat

In [5]:
def train_input_fn():
    return (tf.constant(trainencflt, shape = [trainencflt.shape[0], trainencflt.shape[1]]), None)

In [13]:
seed = 12345
model = KMeansClustering(num_clusters=1000, 
                         initial_clusters=KMeansClustering.RANDOM_INIT, 
                         relative_tolerance=1e-8,
                         random_seed = seed)
t0 = time.time()
model.fit(input_fn = train_input_fn, steps = 100)
t1 = time.time()
print("TF time taken: %r" % (t1 - t0))

## 132s for 100 iters ==> 1320s for 1000 iters

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fde3d81a358>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmps3k3cgi8'}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmps3k3cgi8/model.ckpt.
INFO:tensorflow:loss = 2.65638e+12, step = 1
INFO:tensorflow:Saving checkpoints for 100 into /tmp/tmps3k3cgi8/model.ckpt.
INFO:tensorflow:Loss for final step: 1.36678e+12.
TF time taken: 132.4911835193634


### Note: TensorFlow detects the GPUs, but the GPUs aren't doing much compute
```
2017-06-18 17:01:26.394132: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 0 with properties: 
name: TITAN X (Pascal)
major: 6 minor: 1 memoryClockRate (GHz) 1.531
pciBusID 0000:01:00.0
Total memory: 11.90GiB
Free memory: 10.48GiB
2017-06-18 17:01:26.639874: W tensorflow/stream_executor/cuda/cuda_driver.cc:523] A non-primary context 0x19507e00 exists before initializing the StreamExecutor. We haven't verified StreamExecutor works with that.
2017-06-18 17:01:26.640842: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 1 with properties: 
name: GeForce GTX 1080 Ti
major: 6 minor: 1 memoryClockRate (GHz) 1.582
pciBusID 0000:02:00.0
Total memory: 10.91GiB
Free memory: 10.75GiB
2017-06-18 17:01:26.899156: W tensorflow/stream_executor/cuda/cuda_driver.cc:523] A non-primary context 0x19503f60 exists before initializing the StreamExecutor. We haven't verified StreamExecutor works with that.
2017-06-18 17:01:26.900102: I tensorflow/core/common_runtime/gpu/gpu_device.cc:940] Found device 2 with properties: 
name: GeForce GTX 1080 Ti
major: 6 minor: 1 memoryClockRate (GHz) 1.582
pciBusID 0000:03:00.0
Total memory: 10.91GiB
Free memory: 10.75GiB
2017-06-18 17:01:26.902241: I tensorflow/core/common_runtime/gpu/gpu_device.cc:961] DMA: 0 1 2 
2017-06-18 17:01:26.902254: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 0:   Y Y Y 
2017-06-18 17:01:26.902259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 1:   Y Y Y 
2017-06-18 17:01:26.902263: I tensorflow/core/common_runtime/gpu/gpu_device.cc:971] 2:   Y Y Y 

```

```
Sun Jun 18 17:12:59 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 381.22                 Driver Version: 381.22                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    On   | 0000:01:00.0      On |                  N/A |
|100%   40C    P2    56W / 250W |  11694MiB / 12188MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 0000:02:00.0     Off |                  N/A |
| 90%   33C    P8    20W / 250W |  10622MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  GeForce GTX 108...  On   | 0000:03:00.0     Off |                  N/A |
| 80%   32C    P8    19W / 250W |  10622MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU       PID  Type  Process name                               Usage      |
|=============================================================================|
|    0      2309    G   /usr/lib/xorg/Xorg                             738MiB |
|    0      2747    G   compiz                                         467MiB |
|    0     25319    C   ...yenv/versions/3.6.0/envs/h2oai/bin/python 10335MiB |
|    0     30919    G   ...el-token=798A69B792D7683DCFF8AA9AA8F4DFD9   141MiB |
|    1     25319    C   ...yenv/versions/3.6.0/envs/h2oai/bin/python 10611MiB |
|    2     25319    C   ...yenv/versions/3.6.0/envs/h2oai/bin/python 10611MiB |
+-----------------------------------------------------------------------------+

```